In [5]:
import torch
import torch.nn.functional as F

In [6]:
import math
M = 32


def flashattn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    assert Q.shape == K.shape == V.shape
    assert len(Q.shape) == 2

    K = K.T

    # seq length, inner dim of representations
    N, d = Q.shape

    # of kv vectors per tile
    bc = math.ceil(M/(4*d))
    tc = math.ceil(N/bc)
    K_shared = torch.empty((d, bc), dtype=Q.dtype)
    V_shared = torch.empty((bc, d), dtype=Q.dtype)

    # of q vectors per tile
    br = min(math.ceil(M/(4*d)), d)
    tr = math.ceil(N/br)
    Q_shared = torch.empty((br, d), dtype=Q.dtype)

    # print(f'bc={bc}, br={br}')
    # print(f'tc={tc}, tr={tr}')

    # output matrix
    O = torch.zeros_like(Q)
    O_shared = torch.empty_like(Q_shared)

    # intermediate rowmaxes
    m = torch.full((N,), -torch.inf, dtype=Q.dtype)
    m_shared = torch.empty((br, 1), dtype=Q.dtype)
    # intermediate normalization constants
    l = torch.full((N,), 0, dtype=Q.dtype)
    l_shared = torch.empty((br, 1), dtype=Q.dtype)

    for i in range(tc):
        # load k, v chunks
        # make sure we load in k as its transposed version
        K_shared[:, :] = K[:, i*bc:(i+1)*bc]
        V_shared[:, :] = V[i*bc:(i+1)*bc, :]

        for j in range(tr):
            # load in q, o, m, l
            Q_shared[:, :] = Q[j*br:(j+1)*br, :]
            # if i == 0: print(f'Q: {Q_shared}')
            O_shared[:, :] = O[j*br:(j+1)*br, :]
            m_shared[:, :] = m[j*br:(j+1)*br].unsqueeze(-1)
            l_shared[:, :] = l[j*br:(j+1)*br].unsqueeze(-1)
            
            S = Q_shared @ K_shared

            # get row-wise softmax statistics
            mt = S.max(dim=1).values.reshape(-1, 1)

            Pt = torch.exp(S - mt)
            lt = Pt.sum(dim=1).reshape(-1, 1)

            # compute new statistics
            m_new = torch.max(mt, m_shared)
            l_new = (torch.exp(m_shared - m_new) * l_shared) + (torch.exp(mt - m_new) * lt)


            # update chunk of output
            O_new = (l_shared * torch.exp(m_shared - m_new) * O_shared + torch.exp(mt - m_new) * Pt @ V_shared) / l_new
            O[j*br:(j+1)*br, :] = O_new
            
            m[j*br:(j+1)*br] = m_new.flatten()
            l[j*br:(j+1)*br] = l_new.flatten()

    return O






In [55]:
def dumb_attn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    """equivalent to F.scaled_dot_product_attention(Q, K, V, scale=1)"""
    # P = (Q @ K.T)
    # m = P.max(dim=1).values.reshape(-1, 1)
    # print((P - m).exp())
    # return torch.softmax(Q @ K.T, dim=1) @ V
    return F.scaled_dot_product_attention(Q, K, V, scale=1)


Q = torch.rand((8, 2), dtype=torch.float32, device='cuda')
K = torch.rand((8, 2), dtype=torch.float32, device='cuda')
V = torch.rand((8, 2), dtype=torch.float32, device='cuda')

# Q = torch.randint(1, 9, (2, 2), device='cuda').float()
# K = torch.randint(1, 9, (2, 2), device='cuda').float()
# V = torch.randint(1, 9, (2, 2), device='cuda').float()

# Q = torch.tensor([[0., 1.],
#          [2., 3.]])

# K = torch.tensor([[0., 1.5],
#          [2., 3.]])

# V = torch.tensor([[1., 0.],
#          [0., 1.]])
Q, K, V


(tensor([[0.5405, 0.8691],
         [0.8764, 0.3651],
         [0.6020, 0.1577],
         [0.3892, 0.7547],
         [0.8394, 0.7066],
         [0.1834, 0.8229],
         [0.3026, 0.8099],
         [0.9580, 0.4797]], device='cuda:0'),
 tensor([[0.7769, 0.8157],
         [0.9311, 0.5300],
         [0.3401, 0.5987],
         [0.0919, 0.7284],
         [0.5994, 0.8746],
         [0.9553, 0.6966],
         [0.1698, 0.8951],
         [0.5294, 0.5202]], device='cuda:0'),
 tensor([[0.8813, 0.4181],
         [0.0740, 0.2983],
         [0.1123, 0.9229],
         [0.8420, 0.2798],
         [0.1049, 0.9854],
         [0.4296, 0.1808],
         [0.9028, 0.7446],
         [0.5205, 0.8607]], device='cuda:0'))

In [57]:
from torch.utils.cpp_extension import load
module = load(
    name='m',
    sources=['main.cpp', 'flash_attention.cu'],
    verbose=True
)

Using /home/seb/.cache/torch_extensions/py312_cu121 as PyTorch extensions root...
The input conditions for extension module m have changed. Bumping to version 14 and re-building as m_v14...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/seb/.cache/torch_extensions/py312_cu121/m/build.ninja...
/home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module m_v14...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=m_v14 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/THC -isystem /usr/local/cuda-12.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/seb/CUDA/flash-attention/main.cpp -o main.o 
[2/3] /usr/local/cuda-12.3/bin/nvcc --generate-dependencies-with-compile --dependency-output flash_attention.cuda.o.d -DTORCH_EXTENSION_NAME=m_v14 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem 

Loading extension module m_v14...


In [58]:
# torch.isclose(dumb_attn(Q, K, V), flashattn(Q, K, V))
# dumb_attn(Q, K, V), flashattn(Q, K, V)
dumb_attn(Q, K, V), module.flash_attn(Q, K, V)
# F.scaled_dot_product_attention(Q, K, V, scale=1), flashattn(Q, K, V)
# module.flash_attn(Q, K, V)

44
O_s should be zeros on first iter, than something else (before being multiplied by 0.000000):
[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

O_s should be zeros on first iter, than something else (before being multiplied by 0.735759):
[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

O_s should be zeros on first iter, than something else (before being multiplied by 2.735759):
[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

O_s should be zeros on first iter, than something else (before being multiplied by 4.735759):
[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]

[[0.0000, 0.0000, ]
[0.0000, 0.0000, ]]



(tensor([[0.4790, 0.5707],
         [0.4560, 0.5515],
         [0.4619, 0.5627],
         [0.4827, 0.5761],
         [0.4644, 0.5560],
         [0.4928, 0.5846],
         [0.4875, 0.5800],
         [0.4555, 0.5485]], device='cuda:0'),
 tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]], device='cuda:0'))